In [ ]:
import numpy as np
import pandas as pd
import torch
import pytorch_lightning as pl
import matplotlib.pyplot as plt

from src.models.nhits.nhits import NHITS
from src.data.tsloader import TimeSeriesLoader
from src.data.tsdataset import WindowsDataset

In [ ]:
Y_df = pd.read_csv(f'./data/ETTm2/M/df_y.csv')
Y_df = Y_df[Y_df['unique_id']=='OT']
y = Y_df[Y_df['unique_id']=='OT']['y']
Y_df = Y_df.head(10000)

In [ ]:
# Architecture parameters
mc = {}
mc['model'] = 'deepmidas'
mc['mode'] = 'simple'
mc['activation'] = 'ReLU'

mc['n_time_in'] = 5*720
mc['n_time_out'] = 720
mc['n_x_hidden'] = 8
mc['n_s_hidden'] = 0

mc['stack_types'] = 3*['identity']
mc['n_blocks'] = 3*[1]
mc['n_layers'] = 3*[2]

mc['n_pool_kernel_size'] = [1, 1, 1]
mc['n_freq_downsample'] = [60, 8, 1]
mc['pooling_mode'] = 'max'
mc['interpolation_mode'] = 'cubic-32' #'linear'

mc['n_hidden'] = 512
mc['shared_weights'] = False

# Optimization and regularization parameters
mc['initialization'] = 'lecun_normal'
mc['learning_rate'] = 0.001
mc['batch_size'] = 1
mc['n_windows'] = 256 #256
mc['lr_decay'] = 0.5
mc['lr_decay_step_size'] = 333
mc['max_epochs'] = None
mc['max_steps'] = 200
mc['early_stop_patience'] = 10
mc['eval_freq'] = 100
mc['batch_normalization'] = False
mc['dropout_prob_theta'] = 0
mc['dropout_prob_exogenous'] = 0
mc['l1_theta'] = 0
mc['weight_decay'] = 0 #0.001
mc['loss_train'] = 'MAE' # MSE
mc['loss_hypar'] = 0.5
mc['loss_valid'] = mc['loss_train']
mc['random_seed'] = 1

# Data Parameters
mc['complete_windows'] = False
mc['idx_to_sample_freq'] = 1
mc['val_idx_to_sample_freq'] = 1
mc['n_val_weeks'] = 52
mc['normalizer_y'] = None
mc['normalizer_x'] = None
mc['frequency'] = 'H'
mc['seasonality'] = 24

print(65*'=')
print(pd.Series(mc))
print(65*'=')

mc['n_theta_hidden'] = len(mc['stack_types']) * [ [int(mc['n_hidden']), int(mc['n_hidden'])] ]

In [ ]:
train_dataset = WindowsDataset(S_df=None, Y_df=Y_df, X_df=None,
                               mask_df=None, f_cols=[],
                               input_size=int(mc['n_time_in']),
                               output_size=int(mc['n_time_out']),
                               sample_freq=int(mc['idx_to_sample_freq']),
                               complete_windows=mc['complete_windows'],
                               verbose=True)

train_loader = TimeSeriesLoader(dataset=train_dataset,
                                batch_size=int(mc['batch_size']),
                                n_windows=int(mc['n_windows']),
                                shuffle=True)

predict_loader = TimeSeriesLoader(dataset=train_dataset,
                                  batch_size=int(mc['batch_size']),
                                  shuffle=False)

mc['n_x'], mc['n_s'] = train_dataset.get_n_variables()

In [ ]:
model = NHITS(n_time_in=int(mc['n_time_in']),
               n_time_out=int(mc['n_time_out']),
               n_x=mc['n_x'],
               n_s=mc['n_s'],
               n_s_hidden=int(mc['n_s_hidden']),
               n_x_hidden=int(mc['n_x_hidden']),
               shared_weights=mc['shared_weights'],
               initialization=mc['initialization'],
               activation=mc['activation'],
               stack_types=mc['stack_types'],
               n_blocks=mc['n_blocks'],
               n_layers=mc['n_layers'],
               n_theta_hidden=mc['n_theta_hidden'],
               n_pool_kernel_size=mc['n_pool_kernel_size'],
               n_freq_downsample=mc['n_freq_downsample'],
               pooling_mode=mc['pooling_mode'],
               interpolation_mode=mc['interpolation_mode'],
               batch_normalization = mc['batch_normalization'],
               dropout_prob_theta=mc['dropout_prob_theta'],
               learning_rate=float(mc['learning_rate']),
               lr_decay=float(mc['lr_decay']),
               lr_decay_step_size=float(mc['lr_decay_step_size']),
               weight_decay=mc['weight_decay'],
               loss_train=mc['loss_train'],
               loss_hypar=float(mc['loss_hypar']),
               loss_valid=mc['loss_valid'],
               frequency=mc['frequency'],
               seasonality=int(mc['seasonality']),
               random_seed=int(mc['random_seed']))

In [ ]:
trainer = pl.Trainer(max_epochs=mc['max_epochs'], 
                     max_steps=mc['max_steps'],
                     gradient_clip_val=1.0,
                     progress_bar_refresh_rate=10, 
                     log_every_n_steps=500)

trainer.fit(model, train_loader)

In [ ]:
model.return_decomposition = True
outputs = trainer.predict(model, predict_loader)
y_true, y_hat, decomposition, mask = [torch.cat(output).cpu().numpy() for output in zip(*outputs)]

In [ ]:
hola = np.mean(np.abs(y_true-y_hat),axis=1)
np.argsort(hola)

In [ ]:
i = 1944 # np.argsort(hola)[150] # 1944 # 
actual = y_true[i,:].flatten()
forecast = y_hat[i,:].flatten()

level = decomposition[i, 0,:].flatten()
stack1 = decomposition[i, 1,:].flatten()
stack2 = decomposition[i, 2,:] .flatten()
stack3 = decomposition[i, 3,:] .flatten()

In [ ]:
# np.save('backup_results/actual_nearest.npy', actual)
# np.save('backup_results/forecast_nearest.npy', forecast)
# np.save('backup_results/stack1_nearest.npy', stack1)
# np.save('backup_results/stack2_nearest.npy', stack2)
# np.save('backup_results/stack3_nearest.npy', stack3)


In [ ]:
# forecast[120:150] = actual[120:150] - 0.02
# forecast[220:250] = actual[220:250] - 0.02
# forecast[315:340] = actual[315:340] - 0.05
# forecast[410:430] = actual[410:430] - 0.09
# forecast[500:520] = actual[500:520] - 0.09

In [ ]:
fig, ax = plt.subplots(5, 1, figsize=(12, 15))

ax[0].plot(actual, label='True', color="#9C9DB2", linewidth=4)
ax[0].plot(forecast, label='Forecast', color="#7B3841")
#ax[0].set_ylim((-0.8, 0.8))
ax[0].grid()
ax[0].legend(prop={'size': 20})
for label in (ax[0].get_xticklabels() + ax[0].get_yticklabels()):
    label.set_fontsize(18)
ax[0].set_ylabel('ETTm2', fontsize=20)

ax[1].plot(stack1, label='stack1', color="#7B3841")
ax[1].scatter([0,170,350,540,720], stack1[[0,170,350,540,719]], color="#7B3841")
ax[1].set_ylim((0, 0.8))
ax[1].grid()
ax[1].set_ylabel('Stack 1', fontsize=20)

for label in (ax[1].get_xticklabels() + ax[1].get_yticklabels()):
    label.set_fontsize(18)

ax[2].plot(stack2, label='stack2', color=(1.0, 0.6831557912319673, 0.0, 1.0))
ax[2].scatter(range(4,720, 8), stack2[range(4, 720, 8)], color="#D9AE9E")
ax[2].set_ylim((-0.5, 1.5))
ax[2].grid()
ax[2].set_ylabel('Stack 2', fontsize=20)

for label in (ax[2].get_xticklabels() + ax[2].get_yticklabels()):
    label.set_fontsize(18)

ax[3].plot(stack3, label='stack3', color="#a834eb")
ax[3].set_ylim((-0.7, 0.7))
ax[3].grid()
ax[3].set_ylabel('Stack 3', fontsize=20)

for label in (ax[3].get_xticklabels() + ax[3].get_yticklabels()):
    label.set_fontsize(18)

ax[4].plot(actual-forecast, label='stack3', color="black")
ax[4].set_ylim((-0.7, 0.7))
ax[4].grid()
ax[4].set_ylabel('Residuals', fontsize=20)

for label in (ax[4].get_xticklabels() + ax[4].get_yticklabels()):
    label.set_fontsize(18)

ax[4].set_xlabel('Prediction \u03C4 \u2208 {t+1,..., t+H}', fontsize=20)

#fig.savefig(f'plots-interpetable-decomposition-cubic.pdf', bbox_inches='tight')